In [33]:
# Import libraries
import boto3
import json
import heapq
from heapq import heappop, heappush
from collections import deque
# import itertools
# import pandas as pd
# from io import StringIO

In [2]:
# Set variables for s3 connection
region = 'ap-northeast-2'

bucket = 'emr-data-sync'
prefix = 'segment-logs/pAzAoFnKNpLfHxRmZ2oDF4/'
max_keys = 1000
delimiter = '/'

In [3]:
# S3 onnection with boto3
s3 = boto3.client('s3', region_name=region)
response = s3.list_objects(Bucket=bucket)

objects = s3.list_objects_v2(Bucket=bucket,
                             Prefix=prefix,
                             MaxKeys=max_keys,
                             Delimiter=delimiter)

print(f"Folders Count : {objects['KeyCount']}")
print()
print(f"If \"Folders Count\" is near 1,000 Please Examine \"max_keys\"")
print(f">>> max_keys = {max_keys}")

Folders Count : 145

If "Folders Count" is near 1,000 Please Examine "max_keys"
>>> max_keys = 1000


In [7]:
# Nested folders list to dequeue
folder_prefix_queue = deque()

for obj in objects.get('CommonPrefixes'):
    folder_prefix_queue.appendleft(obj.get('Prefix'))

print(f"Dequeued Folders Count : {len(folder_prefix_queue)}\n")
if objects['KeyCount'] == len(folder_prefix_queue):
    print(f"All Folders Successfully Dequeued")
else:
    print(f"Failed to Deque All Folders.\nPlease Check The Code")

Dequeued Folders Count : 145

All Folders Successfully Dequeued


In [77]:
files_heapq = []

for prefix_deque in folder_prefix_queue:
    ooo = s3.list_objects_v2(Bucket=bucket,
                            Prefix=prefix_deque,
                            MaxKeys=max_keys,
                            Delimiter=delimiter)
    for item in ooo['Contents']:
        files = item['Key']
        heapq.heappush(files_heapq, files)

print(f"HeapQueued Files Count : {len(files_heapq)}")

files_sorted = [heappop(files_heapq) for i in range(len(files_heapq))]

HeapQueued Files Count : 144484


In [86]:
files_sorted[-5:]

['segment-logs/pAzAoFnKNpLfHxRmZ2oDF4/1664755200000/1664809253370.events-archiver-56.16d05ba.e606fcea-67c3-4b66-af90-0a4d8e307244.gz',
 'segment-logs/pAzAoFnKNpLfHxRmZ2oDF4/1664755200000/1664809339252.events-archiver-0.16d05ba.03c8332a-75cb-4fee-85bf-79f2007b7426.gz',
 'segment-logs/pAzAoFnKNpLfHxRmZ2oDF4/1664755200000/1664809350234.events-archiver-32.16d05ba.151d2cc1-cc74-4118-919b-29b597717be3.gz',
 'segment-logs/pAzAoFnKNpLfHxRmZ2oDF4/1664755200000/1664809365411.events-archiver-44.16d05ba.5b987746-6c84-4dc8-a0c5-039468d07ee4.gz',
 'segment-logs/pAzAoFnKNpLfHxRmZ2oDF4/1664755200000/1664809401168.events-archiver-1.16d05ba.41a99a59-b8c6-4af4-9aa0-41b5d7705dc7.gz']